In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

# Definition des fonctions

In [ ]:
def neural_networks():

    model = models.Sequential()

    model.add(layers.Dense(128, activation='relu', input_shape=(100*19,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(24, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


# Construction base de données

In [ ]:
X_train_intitial=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
y_train_intitial=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv") 
X_test_initial=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_test_m4HAPAP.csv") 

In [ ]:
def data_base(X):
    X['bid_ask_spread']=X['ask']-X['bid']
    X['trade']=X['trade'].astype(int)

    value_categor=['venue','action','side']

    for k in value_categor:

        X_encoded = pd.get_dummies(X[k], prefix=k).astype(int)
        X = X.merge( X_encoded , left_index = True , right_index = True)
        X.drop( columns=[k] , inplace = True )

    X.drop(columns=['obs_id','order_id'],inplace=True)


    value_no_center=['price' , 'bid' , 'ask' , 'bid_size', 'ask_size', 'bid_ask_spread' ]

    for k in value_no_center:
    
        X[k] = X[k] / max( X[k] )

    X=X.values.reshape(-1, 19 * 100)
    
    return X

# Test du modèle

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_base(X_train_intitial), y_train_intitial['eqt_code_cat'].array, test_size=0.2, random_state=42)

In [ ]:
model_1 = neural_networks()
model_1.fit(X_train, y_train, epochs=10, batch_size=64)

predictions_prod = model_1.predict(X_test)
y_pred=[]
for liste in predictions_prod:
    top = liste.argmax()
    y_pred += [top]
    
accuracy_score(y_pred,y_test)